In [126]:
import numpy as np
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
import random

In [176]:
def make_regression(n_rows=100,n_features=1, random_state=0):
  np.random.seed(random_state)
  coefs=np.random.randint(0,10,n_features+1)
  X=np.random.normal(0,10,(n_rows,n_features))
  X=np.vstack((np.ones(n_rows),X.T)).T
  y=np.dot(X,coefs)
  return X,y,coefs

In [177]:
class Dataset:
  def __init__(self,n_rows=100,n_cols=1,batch_size=50, random_state=0):
    X,y,coefs=make_regression(n_rows,n_cols,random_state)

    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=random_state)
    self.X_train=X_train
    self.X_test=X_test
    self.y_train=y_train
    self.y_test=y_test
    self.batch_size=batch_size
    self.coefs=coefs

  def minibatch(self):
    for i in range(0,self.X_train.shape[0],self.batch_size):
      yield self.X_train[i:i+self.batch_size],self.y_train[i:i+self.batch_size]

  def plot(self,train=True,test=True,train_pred=None,test_pred=None):
    fig=go.Figure()
    if train:
      fig.add_trace(go.Scatter(x=self.X_train[:,1],y=self.y_train,name="train",mode="markers"))
    if train_pred:
      fig.add_trace(go.Scatter(x=self.X_train[:,1],y=train_pred,name="train predictions",mode="markers"))
    
    if test_pred:
      fig.add_trace(go.Scatter(x=self.X_train[:,1],y=test_pred,name="test_predictions",mode="markers"))
    if test:
      fig.add_trace(go.Scatter(x=self.X_train[:,1],y=self.y_test,name="test",mode="markers"))
    return fig

    

In [178]:
dataset=Dataset(n_rows=1000,batch_size=100,n_cols=5)

In [179]:
fig=dataset.plot()
fig.show()

In [180]:
def MSE(y_true,y_pred):#Mean squared error
  return np.sum((y_true-y_pred)**2)/y_true.shape[0]

def forward(X,theta):
  return np.dot(X,theta)

In [181]:
eps=0.01
num_iters=1000
i=0
lr=0.01

theta=np.zeros(dataset.X_train.shape[1])
J=np.inf
while J>eps and i<=num_iters:
  if i%100==0:
    print(f'iter #{i}, error: {J}')
  i+=1
  for X_batch,y_batch in dataset.minibatch():
    y_batch_pred=forward(X_batch,theta)

    J=MSE(y_batch,y_batch_pred)
    #0
    #2/n sum
    for j in range(len(theta)):
      dtheta=-2/dataset.X_train.shape[0]*np.sum(np.dot(y_batch-y_batch_pred,X_batch[:,j]))
      # dtheta_0=-2/dataset.X_train.shape[0]*np.sum(np.dot(y_batch-y_batch_pred,X_batch[:,0]))
      # dtheta_1=-2/dataset.X_train.shape[0]*np.sum(np.dot(y_batch-y_batch_pred,X_batch[:,1]))
      # theta[0]-=lr*dtheta_0
      # theta[1]-=lr*dtheta_1
      theta[j]-=lr*dtheta
print(f'iter #{i}, error: {J}')
print(theta,dataset.coefs)

iter #0, error: inf
iter #100, error: 0.46348581869746297
iter #197, error: 0.009621812614687263
[4.90315602e+00 1.63582472e-04 2.99995496e+00 3.00040860e+00
 6.99986684e+00 9.00018133e+00] [5 0 3 3 7 9]
